In [202]:
import numpy as np
import tensorflow as tf

In [203]:
#importing the dataset
import pandas as pd
data=pd.read_csv("E:\\datasets\\cancerdatahp-lung-cancer-data\\cancer patient data sets.csv")

In [204]:
data.head()

,Patient Id,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,P1,33,1,2,4,5,4,3,2,2,...,3,4,2,2,3,1,2,3,4,Low
1,P10,17,1,3,1,5,3,4,2,2,...,1,3,7,8,6,2,1,7,2,Medium
2,P100,35,1,4,5,6,5,5,4,6,...,8,7,9,2,1,4,6,7,2,High
3,P1000,37,1,7,7,7,7,6,7,7,...,4,2,3,1,4,5,6,7,5,High
4,P101,46,1,6,8,7,7,7,6,7,...,3,2,4,1,4,2,4,2,3,High


In [205]:
#structuring the dataset for training
##converting the pandas dataset to numpy array
tr_features=data.drop(["Patient Id","Level"],axis=1).values
df=pd.get_dummies(data['Level'])
tr_labels=df.values

In [206]:
#creating class column

In [207]:
data['classes']=0

In [208]:
data['classes'][data['Level']=='High']=0
data['classes'][data['Level']=='Medium']=2
data['classes'][data['Level']=='Low']=1
#classes present in the dataset
classs=pd.DataFrame({
    'High':[0],
    'Medium':[2],
    'Low':[1]
})

c:\users\assistant analystef1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\assistant analystef1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\assistant analystef1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [209]:
training_epochs=500
n_neurons_h1=30
n_neurons_h2=30
learning_rate=0.01
n_features=tr_features.shape[1]
n_classes=3

In [210]:
#constructing the computation graph
X=tf.placeholder(tf.float32,[None,n_features],name='features')
Y=tf.placeholder(tf.float32,[None,n_classes],name='labels')
#layer 1 weights and biases and the output of layer 1
W1=tf.Variable(tf.truncated_normal([n_features,n_neurons_h1],mean=0,stddev=1/np.sqrt(n_features))
               ,name='weights1')
b1=tf.Variable(tf.truncated_normal([n_neurons_h1],mean=0,stddev=1/np.sqrt(n_features)),name='biases1')

y1=tf.nn.tanh((tf.matmul(X,W1)+b1),name='activationLayer1')


#layer 2 weights and biases and the output of layer 2
W2=tf.Variable(tf.truncated_normal([n_neurons_h1,n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features))
               ,name='weights2')
b2=tf.Variable(tf.truncated_normal([n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features)),name='biases2')
y2=tf.nn.tanh((tf.matmul(y1,W2)+b2),name='activationLayer2')

#output layer weights and biasies
Wo = tf.Variable(tf.random_normal([n_neurons_h2, n_classes], mean=0, stddev=1/np.sqrt(n_features))
                 , name='weightsOut')
bo = tf.Variable(tf.random_normal([n_classes], mean=0, stddev=1/np.sqrt(n_features)), name='biasesOut')
#activation function(softmax)
a = tf.nn.softmax((tf.matmul(y2, Wo) + bo), name='activationOutputLayer')

In [211]:
#defining the cost function for training
cross_entropy=tf.reduce_mean(-tf.reduce_sum(Y*tf.log(a),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [212]:
initial=tf.global_variables_initializer()

In [213]:
with tf.Session() as sess:
    sess.run(initial)
    
    batch_size=10
    for epoch in range(training_epochs):
        for i in range(len(tr_features)):
            start=i
            end=i+batch_size
            x_batch=tr_features[start:end]
            y_batch=tr_labels[start:end]
            
            #feeding the training examples
            sess.run(train_step,feed_dict={X:x_batch,Y:y_batch})
            i+=batch_size
            
        #feeding testing data to determine model accuracy
        y_pred=sess.run(tf.argmax(a,1),feed_dict={X:tr_features,Y:tr_labels})
            
        #write results to a summary file
        #writer.add_summary(summary,epoch)
            
        #print accuracy for each epoch
        if (epoch%20==0):
            print("epoch:{}".format(epoch))
            print("===========================================================")

epoch:0
epoch:20
epoch:40
epoch:60
epoch:80
epoch:100
epoch:120
epoch:140
epoch:160
epoch:180
epoch:200
epoch:220
epoch:240
epoch:260
epoch:280
epoch:300
epoch:320
epoch:340
epoch:360
epoch:380
epoch:400
epoch:420
epoch:440
epoch:460
epoch:480


array([[365,   0,   0],
       [  0, 303,   0],
       [  0,   0, 332]], dtype=int64)

In [215]:
import pandas as pd
y_true = pd.Series(y_pred)
y_pred = pd.Series(data['classes'])

In [216]:
classs

,High,Low,Medium
0,0,1,2


In [217]:
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,All
True,,,,
0,365,0,0,365
1,0,303,0,303
2,0,0,332,332
All,365,303,332,1000


In [218]:
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'],
            margins=True).apply(lambda r: r/r.sum(), axis=1)

Predicted,0,1,2,All
True,,,,
0,0.5000,0.0000,0.000,0.5
1,0.0000,0.5000,0.000,0.5
2,0.0000,0.0000,0.500,0.5
All,0.1825,0.1515,0.166,0.5
